# $\Delta$ Class Development
Interface notebook testing a Python class created from `advanced_profile_creation-Dev`

### Author
John Wall (jwall@Dewberry.com)

### Import libraries

In [1]:
from TransitOps import *

### Provide input data

In [2]:
streams = "/mnt/c/gis/fcast_data/sample_streams.shp"
roads = "/mnt/c/gis/fcast_data/sample_roads.shp"
dem = "/mnt/c/gis/fcast_data/tiffs/c67567_aoi.tif"

our_prj = {'proj': 'aea', 'lat_1': 20, 'lat_2': 60, 'lat_0': 40,
             'lon_0': -96, 'x_0': 0, 'y_0': 0, 'ellps': 'GRS80',
             'units': 'm', 'no_defs': True}

### Test class object initialization

In [3]:
datasets = DepthToInundationPts(streams, roads, dem, our_prj)

### Test object output

In [4]:
datasets.delta_z

,transit,geometry,geometry_min,side,offset,offset_min,delta_z
0,Besstown Rd,LINESTRING Z (1264274.342034306 -452328.815437...,226.710168,upstream,LINESTRING Z (1264180.085532356 -452295.715640...,226.356918,0.353249
1,Costner School Rd,LINESTRING Z (1266126.206046507 -452609.044009...,223.999378,upstream,LINESTRING Z (1266069.092688204 -452526.958295...,222.211342,1.788036
2,Puetts Chapel Rd,LINESTRING Z (1264972.305214914 -452754.188909...,228.069603,upstream,LINESTRING Z (1264879.552337602 -452716.813734...,224.987543,3.082060
3,Besstown Rd,LINESTRING Z (1264274.342034306 -452328.815437...,226.710168,downstream,LINESTRING Z (1264418.469588207 -452152.987224...,226.411835,0.298333
4,Costner School Rd,LINESTRING Z (1266126.206046507 -452609.044009...,223.999378,downstream,LINESTRING Z (1266346.404820265 -452520.689647...,221.328263,2.671115
5,Puetts Chapel Rd,LINESTRING Z (1264972.305214914 -452754.188909...,228.069603,downstream,LINESTRING Z (1265133.880477114 -452597.678766...,224.766019,3.303584
